**Mount Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Import Libraries**

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

**Load Dataset**

In [3]:
# Load and preprocess the data
df = pd.read_csv('Heart.csv')
X = df.drop(columns=['target'])
y = df['target']

print(X.shape), print(y.shape)

(303, 13)
(303,)


(None, None)

In [4]:
print(X[:5])
print(y[:5])

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   

   ca  thal  
0   0     1  
1   0     2  
2   0     2  
3   0     2  
4   0     2  
0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64


In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((242, 13), (61, 13), (242,), (61,))

In [6]:
# Further split the training data to create a validation set for blending
X_train_blend, X_val_blend, y_train_blend, y_val_blend = train_test_split(X_train, y_train, test_size=0.2, random_state=8)
X_train_blend.shape, X_val_blend.shape, y_train_blend.shape, y_val_blend.shape

((193, 13), (49, 13), (193,), (49,))

In [7]:
# Define base models for blending
base_models = {
    'rf': RandomForestClassifier(n_estimators=10, random_state=42),
    'knn': KNeighborsClassifier(n_neighbors=10),
    'gbdt': GradientBoostingClassifier()
}

In [8]:
# Train base models
base_model_predictions = []
for name, model in base_models.items():
    model.fit(X_train_blend, y_train_blend)
    base_model_predictions.append(model.predict_proba(X_val_blend))

In [9]:
# Stack predictions to create the blending dataset
blending_X = np.column_stack([preds[:, 1] for preds in base_model_predictions])
blending_y = y_val_blend

In [10]:
# Train the meta-model on the blending dataset
meta_model = LogisticRegression()
meta_model.fit(blending_X, blending_y)

LogisticRegression()

In [11]:
# Predict on the test set using the blending model
test_base_model_predictions = []
for name, model in base_models.items():
    test_base_model_predictions.append(model.predict_proba(X_test))

test_blending_X = np.column_stack([preds[:, 1] for preds in test_base_model_predictions])
blending_predictions = meta_model.predict(test_blending_X)

In [12]:
# Evaluate the blending model
blending_accuracy = accuracy_score(y_test, blending_predictions)
print(f"Accuracy of Blending Classifier: {blending_accuracy:.4f}")

Accuracy of Blending Classifier: 0.8361
